In [1]:
import reservoirpy as rpy

rpy.verbosity(0)
rpy.set_seed(500)

In [2]:
from itertools import product

size = [2**i for i in range(13, 14)]
# size = [200]
lr = [0.2 * i for i in range(4, 6)]
sr = [0.5 * i for i in range(1, 5)]
# input_scaling = [10**i for i in range(-1, 2)]
input_scaling = [1.0, 10.0]
ridge = [10 ** -i for i in range(5, 10)]

combinations = list(product(size, lr, sr, input_scaling, ridge))
len(combinations)

80

In [3]:
from reservoirpy.nodes import Reservoir, Ridge
from main import samples, evaluate
import numpy as np
import matplotlib.pyplot as plt

diffs = {}
zz = None
zz_ = None
for combination in combinations:
    size, lr, sr, input_scaling, ridge = combination
    ridge = Ridge(ridge=ridge)
    reservoir = Reservoir(
        size,
        lr=lr,
        sr=sr,
        input_scaling=input_scaling,
        fb_connectivity=0.1,
        input_connectivity=0.1,
    )
    diff = None
    for x, y, z in samples():
        z_ = evaluate(x, y, reservoir=reservoir, ridge=ridge, warmup=20)
        if zz_ is None:
            zz_ = z_
        else:
            zz_ = np.concatenate((zz_, z_), axis=0)
        if zz is None:
            zz = z
        else:
            zz = np.concatenate((zz, z), axis=0)
        if diff is None:
            diff = abs(z - z_)
        else:
            diff = np.concatenate(abs(z - z_), axis=0)
    diffs[combination] = diff

m = [a.mean() for a in diffs.values()]
s = [a.std() for a in diffs.values()]

plt.xlabel("$combination$")
plt.plot(m, color="blue", alpha=0.5, label="mean")
plt.plot(s, color="red", alpha=0.5, label="std")
plt.show()

/Users/conrad/Projects/conradbzura/ml/.venv/lib/python3.13/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=1.81279e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/conrad/Projects/conradbzura/ml/.venv/lib/python3.13/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=1.81327e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/conrad/Projects/conradbzura/ml/.venv/lib/python3.13/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=1.81375e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/conrad/Projects/conradbzura/ml/.venv/lib/python3.13/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=2.38911e-17): result may not be accurate.
  return linalg.solve(XXT 

KeyboardInterrupt: 

In [ ]:
m_ = m.copy()

m_sorted = sorted(enumerate(m_), key=lambda x: x[1])

s_ = s.copy()

s_sorted = sorted(enumerate(s_), key=lambda x: x[1])

plt.xlabel("$combination$")
plt.plot([x[1] for x in m_sorted], color="blue", alpha=0.5, label="mean")
plt.plot([x[1] for x in s_sorted], color="red", alpha=0.5, label="std")
plt.show()

print(f"{m_sorted[0]} +/- {s_sorted[0]}")

#### Prepare input data

In [ ]:
print(z_, z)